In [1]:
from Project4.verification_net import VerificationNet
from Project4.stacked_mnist import StackedMNISTData, DataMode
from var_ae_routine import VAE_Routine
import torch

In [ ]:
torch.manual_seed(1)
""" GENERATIVE VAE ROUTINE"""
batch_size = 16
data_object = StackedMNISTData(
    mode=DataMode.MONO_FLOAT_COMPLETE,
    default_batch_size=batch_size)
net = VerificationNet(force_learn=False)
net.train(
    generator=data_object,
    epochs=5)  # gen=data_object, makes sure we test on the same type of data as the model was trained on
verification_tolerance = 0.8 if data_object.channels == 1 else 0.5

learning_rate = 1.0e-1
loss_function = 'elbo'
optimizer = 'adam'
epochs = 20

latent_vector_size = 256
num_samples = 2000
gen_name = 'High_lr'
gen_vae_save_path = f'checkpoints/gen_VAE/{gen_name}'
vae_routine = VAE_Routine(
    data_object,
    learning_rate,
    loss_function,
    optimizer,
    epochs,
    #
    latent_vector_size,
    batch_size,
    num_samples,
    gen_vae_save_path
)

vae_routine.train_vae()

#Note, returned images, reconstructions and gen images are np arrays
images, reconstructions, labels = vae_routine.reconstruct_test_data()
# Check quality of reconstructions:
print('CHECKING RECONSTRUCTED IMAGES QUALITY')
print(f'Number of reconstructions: {len(reconstructions)}')
vae_routine.check_vae_performance(net, verification_tolerance, reconstructions, labels)
# Check quality of generated images
print('CHECKING GENERATED IMAGES QUALITY')
generated_images = vae_routine.generate_samples()
print(f'Number of reconstructions: {len(generated_images)}')
vae_routine.check_vae_performance(net, verification_tolerance, generated_images)

VAE(
  (encoder): Encoder(
    (body): Sequential(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(3, 3), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Conv2d(32, 16, kernel_size=(3, 3), stride=(3, 3), padding=(1, 1))
      (3): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (4): Conv2d(16, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (5): ReLU()
      (6): Flatten(start_dim=1, end_dim=-1)
    )
    (mean_layer): Linear(in_features=128, out_features=256, bias=True)
    (log_std_layer): Linear(in_features=128, out_features=256, bias=True)
  )
  (decoder): Decoder(
    (model): Sequential(
      (0): Linear(in_features=256, out_features=128, bias=True)
      (1): Unflatten(dim=1, unflattened_size=(8, 4, 4))
      (2): ConvTranspose2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): LeakyReLU(negative_slope=0.01)
      (4): BatchNorm2d(16, 